In [1]:
import os, sys
import torch
from torch.utils.data import DataLoader, TensorDataset
import datasets
import evaluate
from datasets import load_dataset, load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    DistilBertForSequenceClassification
)
from peft import PeftModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType, AutoPeftModelForSequenceClassification, PeftConfig, PeftMixedModel, AdaLoraConfig
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-12 13:30:05.028706: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 13:30:06.729018: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 13:30:06.729059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 13:30:06.730185: E external/l

In [2]:
pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from openpyxl import Workbook

In [5]:
model_path = "./distilbert/"
#peft_path = "./distilbert-lora-judge/"
peft_path = "./outputs03121/checkpoint-29000/ada_adapter/"

base_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(base_model, peft_path)
#model = model.merge_and_unload()
model.cuda()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): AdaLoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): adalora.SVDLinear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.01, inplace=False)
                  )
                  (lora_A): ParameterDict(  (default): Parameter containing: [torc

In [6]:
data_dir = "./dataset"   #Bohrium数据集：Finetune-dataset-LLMKG
dataset = load_dataset(
    "json", 
    data_files = {'test': os.path.join(data_dir, 'test_data.jsonl')}
    )

Generating test split: 50000 examples [00:00, 357996.73 examples/s]


In [16]:
'''
#hard prompt
def get_prompt(instruction, inputs):
    #你只能回答问题一次，并确保只回答positive或negative。另外，在生成新的一行前，添加'\n'来标记。如果你没有按照我的要求生成回答，我会继续提问。
    
    prompt = "事件指引: {instruction}\
请仔细阅读以下文本内容，并判断是否包含针对上述事件的投诉或举报信息。\
具体内容: {input}".format(instruction=instruction, input=inputs)
    
    #prompt = "请根据以下任务指引和具体信息描述，判断是否为对某个群体或特定事件的投诉或举报信息。具体的任务指引是{}，信息是{}" .format(instruction,inputs)
    return prompt
def tokenizer_func(example):
    #example["label"] = [int(item) for item in example["label"]]
    #print(example["category_description"])
    prompt = get_prompt(example["category_description"], example["text"])
    return tokenizer(prompt, padding="max_length", truncation="longest_first", max_length=512)

In [7]:
def tokenizer_func(example):
    return tokenizer(example["category_description"], example["text"], padding="max_length", truncation="longest_first", max_length=492)

In [8]:
test_dataset = dataset["test"].map(tokenizer_func, batched=True)
test_dataset = test_dataset.remove_columns(["category_description","text"])
test_dataset.set_format("torch")

Map: 100%|██████████| 50000/50000 [00:08<00:00, 5962.32 examples/s]


In [ ]:
print(tokenizer.decode(test_dataset[0]["input_ids"]))

In [9]:
test_dataloader = DataLoader(test_dataset, batch_size=64, pin_memory=True)
@torch.no_grad()
def eval(inputs):
    #inputs = tokenizer(prompt, return_tensors="pt")
    inputs["input_ids"]=inputs["input_ids"].cuda()
    inputs["attention_mask"]=inputs["attention_mask"].cuda()
    logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    return predicted_class

In [10]:
def excel_write(output_file):
    out_excel = []
    for inputs in tqdm(test_dataloader):
        output = eval(inputs).cpu().numpy().tolist()
        out_excel.append(output)
    wb = Workbook()
    ws = wb.active
    for sublist in out_excel:
        for item in sublist:
            ws.append([item])
    wb.save(output_file)

In [ ]:
output_file = "output.xlsx"
excel_write(output_file)

 88%|████████▊ | 689/782 [01:53<00:15,  6.14it/s]